In [11]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 

- 공항철도 추출

In [33]:
df = pd.read_csv('../../../main_code/data/merged_lines.csv')
df.head()

,사용월,호선명,지하철역,출근시간 승차인원,출근시간 하차인원,09-16시 승차인원,09-16시 하차인원,퇴근시간 승차인원,퇴근시간 하차인원,새벽 승차인원,새벽 하차인원,야간 승차인원,야간 하차인원,총 승차인원,총 하차인원
0,201501,1호선,가능,68085,23287,96744,66805,31775,60948,8225,1240,17566,55647,190620,146979
1,201501,1호선,가산디지털단지,44112,230457,107638,189559,209852,86319,8884,1544,114317,58461,274951,480021
2,201501,1호선,간석,62525,15467,78718,55206,28721,55488,5554,1282,17595,56137,164392,128092
3,201501,1호선,개봉,217770,52104,280814,191890,90855,217153,15614,3593,50829,184806,565027,432393
4,201501,1호선,관악,83780,22169,100507,83845,40644,65829,6103,1307,19637,61074,210027,168395


In [34]:
df = df[df['호선명']== '공항철도']
df.head()

,사용월,호선명,지하철역,출근시간 승차인원,출근시간 하차인원,09-16시 승차인원,09-16시 하차인원,퇴근시간 승차인원,퇴근시간 하차인원,새벽 승차인원,새벽 하차인원,야간 승차인원,야간 하차인원,총 승차인원,총 하차인원
53323,201501,공항철도,검암,179781,34390,183543,91663,60979,136567,14148,1047,27735,167758,405207,294858
53324,201501,공항철도,계양,131009,12294,116463,49635,28113,97189,7878,579,12342,112409,267692,174917
53325,201501,공항철도,공덕,9724,10658,15175,14166,10573,12744,1049,140,7552,11409,33500,36373
53326,201501,공항철도,공항화물청사,5348,42949,13697,18781,34493,3908,669,393,11860,4089,31574,66212
53327,201501,공항철도,김포공항,18142,46070,92992,74109,66978,32247,1979,1500,47812,14709,160925,136388


In [35]:
df = df[['지하철역']]
df.to_csv('./air.csv', index=False)
df = pd.read_csv('./air.csv')
df.head()

,지하철역
0,검암
1,계양
2,공덕
3,공항화물청사
4,김포공항


In [36]:
df = pd.read_csv('./air_test.csv')
df.head()

,지하철역,도로명주소
0,검암,인천광역시 서구 검바위로 26 (검암동)
1,계양,인천광역시 계양구 다남로 24 (귤현동)
2,공덕,서울특별시 마포구 마포대로 92 (도화동 25-13번지)
3,공항화물청사,인천광역시 중구 공항동로135번길 86 (운서동)
4,김포공항,서울특별시 강서구 하늘길 지하 77 (방화동 744번지)


- 주소 확인

In [ ]:
# air_test.csv에 도로명 주소 추가
# 검암,인천광역시 서구 검바위로 26 (검암동)
# 계양,인천광역시 계양구 다남로 24 (귤현동)
# 공덕,서울특별시 마포구 마포대로 92 (도화동 25-13번지)
# 공항화물청사,인천광역시 중구 공항동로135번길 86 (운서동)
# 김포공항,서울특별시 강서구 하늘길 지하 77 (방화동 744번지)
# 디지털미디어시티,서울특별시 마포구 성암로 지하 184 (상암동 1140번지)
# 서울,서울특별시 용산구 청파로 378 (동자동 43-205번지)
# 운서,인천광역시 중구 흰바위로59번길 26 (운서동)
# 인천국제공항,인천광역시 중구 공항로 272 (운서동)
# 청라국제도시,인천광역시 서구 에코로 151 (청라동)
# 홍대입구,서울특별시 마포구 양화로 지하188 (동교동 166-1번지)

In [37]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('../data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [38]:
# 2 입력받은 데이터 프레임의 인덱스로 도로명주소 추출하는 함수
def rtn_addr(target):
    str_addr = df[df.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [39]:
# 3 2번 함수들 사용해서 좌표추가
temp1 =[]
for i in df.index:
    try:
        target = df['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, df.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

In [40]:
kakao_location('서울특별시 마포구 양화로 지하188 (동교동 166-1번지)')

(37.5574452272604, 126.926778735002)

In [41]:
df2.head()

,지하철역,도로명주소,lat,lng
0,검암,인천광역시 서구 검바위로 26 (검암동),37.569030,126.673542
1,계양,인천광역시 계양구 다남로 24 (귤현동),37.571609,126.736023
2,공덕,서울특별시 마포구 마포대로 92 (도화동 25-13번지),37.542844,126.951336
3,공항화물청사,인천광역시 중구 공항동로135번길 86 (운서동),37.459031,126.477438
4,김포공항,서울특별시 강서구 하늘길 지하 77 (방화동 744번지),37.562313,126.801278


In [42]:
df2.to_csv('air_final.csv', index=False)

stn_r_addr_final.csv에 air_final.csv 더하기

In [12]:
df_adfn = pd.read_csv('../stn_r_addr_final.csv')
df_arfn = pd.read_csv('./air_final.csv')

In [13]:
df_adfn.head()

,지하철역,도로명주소,lat,lng
0,계양,인천광역시 계양구 다남로 24,37.571609,126.736023
1,귤현,인천광역시 계양구 장제로 1136,37.566413,126.742649
2,박촌,인천광역시 계양구 장제로 992,37.554143,126.745539
3,임학,인천광역시 계양구 장제로 875,37.545062,126.738623
4,계산,인천광역시 계양구 경명대로 1089,37.543290,126.728081


In [14]:
df_arfn.head()

,지하철역,도로명주소,lat,lng
0,검암,인천광역시 서구 검바위로 26 (검암동),37.569030,126.673542
1,계양,인천광역시 계양구 다남로 24 (귤현동),37.571609,126.736023
2,공덕,서울특별시 마포구 마포대로 92 (도화동 25-13번지),37.542844,126.951336
3,공항화물청사,인천광역시 중구 공항동로135번길 86 (운서동),37.459031,126.477438
4,김포공항,서울특별시 강서구 하늘길 지하 77 (방화동 744번지),37.562313,126.801278


In [7]:
df_adfn.shape, df_arfn.shape

((635, 4), (11, 4))

In [16]:
df_res = pd.concat([df_adfn, df_arfn], axis=0)
df_res.head()

,지하철역,도로명주소,lat,lng
0,계양,인천광역시 계양구 다남로 24,37.571609,126.736023
1,귤현,인천광역시 계양구 장제로 1136,37.566413,126.742649
2,박촌,인천광역시 계양구 장제로 992,37.554143,126.745539
3,임학,인천광역시 계양구 장제로 875,37.545062,126.738623
4,계산,인천광역시 계양구 경명대로 1089,37.543290,126.728081


In [8]:
df_res.shape

(646, 4)

In [17]:
df_res.to_csv('../addr_air.csv', index=False)